In [71]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS

In [72]:
with open("orange_qa_database.txt",encoding = 'utf-8') as f:
    orange_sales_vector_database = f.read()

In [73]:
text_splitter = CharacterTextSplitter(        
    separator = '---',
    chunk_size = 200,
    chunk_overlap  = 0,
    length_function = len,
)

In [74]:
docs = text_splitter.create_documents([orange_sales_vector_database])

In [75]:
db = FAISS.from_documents(docs, OpenAIEmbeddings())

In [76]:
query = "有哪些橘子"

In [77]:
answer_list = db.similarity_search(query)

In [78]:
answer_list

[Document(page_content='[问题]  \n你们的橘子都有哪些品种？  \n\n[回答]  \n我们主要有两个品种的橘子，分别是甘平和爱媛。甘平橘子果肉细嫩、汁水丰富，口感甜美。爱媛橘子则皮薄肉多，风味清甜，且香气浓郁。无论是鲜食还是榨汁，都是非常好的选择。'),
 Document(page_content='[问题]  \n除了爱媛橘子，还有其他果子吗？  \n\n[回答]  \n12月底我们会有甘平橘子上市，这种橘子个头大、果肉饱满，特别适合逢年过节送礼。您吃过甘平吗？它的口感非常棒，是非常受欢迎的品种。'),
 Document(page_content='[问题]  \n你们的橘子能用来做果酱或者甜点吗？  \n\n[回答]  \n是的，甘平和爱媛橘子都非常适合用来做果酱或者甜点。它们的果肉香甜，水分充足，做出来的果酱口感浓郁。而且橘子的天然甜味可以减少额外的糖分添加，是健康又美味的选择！'),
 Document(page_content='[问题]  \n橘子酸吗？我比较怕酸。  \n\n[回答]  \n橘子的酸甜度可能会受到光照等自然条件的影响，但我们家的橘子没有使用甜蜜素等催熟剂，所以口感是天然的、最好的。爱媛橘子通常比较甜，甘平橘子的甜度也很高。您可以先买一箱试试看，如果觉得合适再购买更多。\n\n---\n\n[问题]  \n橘子什么时候成熟的？  \n\n[回答]  \n我们的橘子10月底就成熟了哦，现在正是最好的时候，赶紧入手品尝吧！')]

In [79]:
db.save_local("orange_sales_vector_database")

In [80]:
import gradio as gr
print(gr.themes)

<module 'gradio.themes' from 'E:\\Programing\\Anaconda3\\envs\\langchain\\lib\\site-packages\\gradio\\themes\\__init__.py'>


In [12]:
import os
from langchain.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.prompts import MessagesPlaceholder
from langchain.chains import create_retrieval_chain
from langchain.chains import create_history_aware_retriever
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
os.environ['HTTP_PROXY'] = '127.0.0.1:7890'
os.environ['HTTPS_PROXY'] = '127.0.0.1:7890'

In [81]:
db = FAISS.load_local("orange_sales_vector_database", OpenAIEmbeddings(),allow_dangerous_deserialization = True)
retriever=db.as_retriever(search_type="similarity_score_threshold",
                          search_kwargs={"score_threshold": 0.8})
llm = ChatOpenAI(model="gpt-4o-mini",
                openai_proxy = 'http://127.0.0.1:7890')

In [13]:
### Contextualize question ###
contextualize_q_system_prompt = (
    "Given a chat history and the latest user question "
    "which might reference context in the chat history, "
    "formulate a standalone question which can be understood "
    "without the chat history. Do NOT answer the question, "
    "just reformulate it if needed and otherwise return it as is."
)
contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)
history_aware_retriever = create_history_aware_retriever(
    llm, retriever, contextualize_q_prompt
)


In [14]:
### Answer question ###
system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)
qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)
question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)

rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)

In [15]:
### Statefully manage chat history ###
store = {}


def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]


conversational_rag_chain = RunnableWithMessageHistory(
    rag_chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
    output_messages_key="answer",
)

In [17]:
query = "小区吵不吵？"
conversational_rag_chain.invoke(
    {"input": query},
    config={
        "configurable": {"session_id": "abc123"}
    },  # constructs a key "abc123" in `store`.
)

{'input': '小区吵不吵？',
 'chat_history': [HumanMessage(content='小区吵不吵？'),
  AIMessage(content='这个小区特别注重居住体验，拥有良好的隔音设计，并且内部规划了绿化区域，可以有效降低噪音。')],
 'context': [Document(page_content='[客户问题] 我担心楼下太吵。\n[销售回答] 这个小区特别注重居住体验，我们有良好的隔音设计，并且小区内部规划了绿化区域，可以有效降低噪音。'),
  Document(page_content='[客户问题] 我喜欢安静，这里噪音大吗？\n[销售回答] 我们特意进行了隔音设计，并且小区内部也有绿化带，整体非常安静。'),
  Document(page_content='[客户问题] 我担心楼下的商业活动会很吵。\n[销售回答] 我们在规划时就已经考虑到这一点，商业区和居住区有一定的距离和隔音设计。')],
 'answer': '小区经过特别的隔音设计，并且内部有绿化带，整体非常安静。'}

In [18]:
query = "那交通呢？"
conversational_rag_chain.invoke(
    {"input": query},
    config={
        "configurable": {"session_id": "abc123"}
    },  # constructs a key "abc123" in `store`.
)

{'input': '那交通呢？',
 'chat_history': [HumanMessage(content='小区吵不吵？'),
  AIMessage(content='这个小区特别注重居住体验，拥有良好的隔音设计，并且内部规划了绿化区域，可以有效降低噪音。'),
  HumanMessage(content='小区吵不吵？'),
  AIMessage(content='小区经过特别的隔音设计，并且内部有绿化带，整体非常安静。')],
 'context': [Document(page_content='[客户问题] 这个小区交通便利吗？\n[销售回答] 当然了，这个小区距离地铁站只有几分钟的步行距离，而且附近有多条公交线路，非常方便。'),
  Document(page_content='[客户问题] 我听说这个区域交通常常堵塞。\n[销售回答] 我们小区的位置实际上是绕开了主要交通瓶颈区，所以相对来说会流畅很多。'),
  Document(page_content='[客户问题] 交通如何？\n[销售回答] 交通非常便利，不仅靠近地铁站，还有多条公交线路。'),
  Document(page_content='[客户问题] 我担心小区会很拥挤。\n[销售回答] 这个小区总体规划非常合理，保证了每个单元之间有足够的空间。')],
 'answer': '交通非常便利，小区距离地铁站只有几分钟的步行距离，还有多条公交线路。'}

In [19]:
query = "环境如何？"
conversational_rag_chain.invoke(
    {"input": query},
    config={"configurable": {"session_id": "abc123"}}, 
)

{'input': '环境如何？',
 'chat_history': [HumanMessage(content='小区吵不吵？'),
  AIMessage(content='这个小区特别注重居住体验，拥有良好的隔音设计，并且内部规划了绿化区域，可以有效降低噪音。'),
  HumanMessage(content='小区吵不吵？'),
  AIMessage(content='小区经过特别的隔音设计，并且内部有绿化带，整体非常安静。'),
  HumanMessage(content='那交通呢？'),
  AIMessage(content='交通非常便利，小区距离地铁站只有几分钟的步行距离，还有多条公交线路。')],
 'context': [Document(page_content='[客户问题] 小区里有公园吗？\n[销售回答] 是的，我们有专门设计的绿化区和儿童游乐园，确保您和家人能享受到高质量的生活。'),
  Document(page_content='[客户问题] 我喜欢安静，这里噪音大吗？\n[销售回答] 我们特意进行了隔音设计，并且小区内部也有绿化带，整体非常安静。'),
  Document(page_content='[客户问题] 我很关注环境保护，你们有做到吗？\n[销售回答] 非常好的问题，我们的建筑采用环保材料，并且在小区规划中也有绿化区域和垃圾分类系统。'),
  Document(page_content='[客户问题] 我担心楼下太吵。\n[销售回答] 这个小区特别注重居住体验，我们有良好的隔音设计，并且小区内部规划了绿化区域，可以有效降低噪音。')],
 'answer': '小区环境优美，采用了环保材料，并设有绿化区域，提供高质量的居住环境。'}

In [29]:
qa_prompt

ChatPromptTemplate(input_variables=['chat_history', 'context', 'input'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, say that you don't know. Use three sentences maximum and keep the answer concise.\n\n{context}")), MessagesPlaceholder(variable_name='chat_history'), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}'))])

In [3]:
llm = ChatOpenAI(model="gpt-4o-mini",
                openai_proxy = 'http://127.0.0.1:7890')

system_prompt = (
    "Use the given context to answer the question. "
    "If you don't know the answer, say you don't know. "
    "Use three sentence maximum and keep the answer concise. "
    "Context: {context}"
)
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

In [4]:
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [5]:
query = "小区吵不吵？"
rag_chain.invoke({"input": query})

{'input': '小区吵不吵？',
 'context': [Document(page_content='[客户问题] 我担心楼下太吵。\n[销售回答] 这个小区特别注重居住体验，我们有良好的隔音设计，并且小区内部规划了绿化区域，可以有效降低噪音。')],
 'answer': '小区特别注重居住体验，拥有良好的隔音设计和绿化区域，可以有效降低噪音。'}